### Summary

The idea here is to design a dnn prediction algorithm using tensorflow. We will be using estimator API, which allows us seamless transition to other models !

In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error
import shutil

/Users/babs4JESUS/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
# The directory for storing metadata related to the model. This way, we can easily build models step by step, starting
# from the previous checkpoint, in cases when we have large amount of data.
OUTDIR = 'sample_model_metadata'

In [13]:
#INPUT_DIR = 'input/'
INPUT_DIR = '../input/competitive-data-science-predict-future-sales/'

In [4]:
#  The pandas input function which is to be fed to the estimator API. NOTE : Since the syntax is very similar, we could 
# have used one function for training and evaluation as well, but we have decided to keep them separately for the sake
# of clarity.
def make_train_input_fn(df, num_epochs=1, shuffle=True):
    return tf.estimator.inputs.pandas_input_fn(
        x=df,
        y=df['item_cnt_month'],
        batch_size=128,
        num_epochs=num_epochs,
        shuffle=shuffle,
        queue_capacity=2000)

def make_eval_input_fn(df):
    return tf.estimator.inputs.pandas_input_fn(
        x=df,
        y=df['item_cnt_month'],
        batch_size=128,
        num_epochs=1,
        shuffle=False,
        queue_capacity=2000)

def make_prediction_input_fn(df):
    return tf.estimator.inputs.pandas_input_fn(
        x=df,
        batch_size=128,
        num_epochs=1,
        shuffle=False,
        queue_capacity=2000)

In [5]:
REL_PREDICTOR_COLS = ['date_block_num', 'shop_id', 'item_id', 'item_category_id']

In [6]:
# This routine makes the input in a tensorflow digestible form.
def make_feature_cols():
  input_columns = [tf.feature_column.numeric_column(k) for k in REL_PREDICTOR_COLS]
  return input_columns


##### Get relevant data into pandas dataframes.

In [14]:
sales_train = pd.read_csv(INPUT_DIR + "sales_train.csv")

In [15]:
items = pd.read_csv(INPUT_DIR + 'items.csv')

In [16]:
item_id_to_item_category_id = dict(zip(items.item_id, items.item_category_id))

In [17]:
monthly_sales_data = sales_train[['date_block_num', 'shop_id', 'item_id', 'item_cnt_day' ]].groupby(
    ['date_block_num', 'shop_id', 'item_id']).sum()
monthly_sales_data.rename(columns={'item_cnt_day':'item_cnt_month'}, inplace=True)
monthly_sales_data.reset_index(['date_block_num', 'shop_id', 'item_id'], inplace=True)

In [18]:
monthly_sales_data['item_category_id'] = monthly_sales_data['item_id'].apply(lambda x : item_id_to_item_category_id.get(x, -1))

In [19]:
monthly_sales_data.columns

Index(['date_block_num', 'shop_id', 'item_id', 'item_cnt_month',
       'item_category_id'],
      dtype='object')

In [23]:
from itertools import product
import time

In [24]:
def get_master_data_shop_id_item_id_corr_month():
    master_data = pd.DataFrame(columns=['date_block_num', 'shop_id', 'item_id'])
    for i in range(sales_train.date_block_num.max()+1):
        shop_id_item_ids = list(product(sales_train[sales_train.date_block_num==i].shop_id.unique(), 
                                        sales_train[sales_train.date_block_num==i].item_id.unique()))
        
        simple_df = pd.DataFrame(shop_id_item_ids, columns=['shop_id', 'item_id'])
        simple_df['date_block_num'] = i
        master_data = pd.concat([master_data, simple_df], sort=True)
    return master_data    

In [25]:
ts = time.time()
master_data = get_master_data_shop_id_item_id_corr_month()
time.time() - ts

16.662301063537598

In [26]:
len(master_data)

10913850

#### Concatenating with a test set.

Now, that we have a master dataframe( and have encapsulated the methods so that we can switch between them easily), let us concate nate them with the test set.

In [27]:
test_data['date_block_num'] = 34

In [28]:
master_data_with_test = pd.concat([master_data, test_data[['date_block_num', 'shop_id', 'item_id']]], sort=True)

In [29]:
len(master_data_with_test)

11128050

In [34]:
master_data_with_test.columns

Index(['date_block_num', 'item_id', 'shop_id'], dtype='object')

#### Now , let us start filling the dataframe with relevant feature data.

In [36]:
master_data_with_test['item_category_id'] = master_data_with_test.item_id.apply(
    lambda x : item_id_to_item_category_id.get(x, 0))

#### Add item_cnt_month

In [37]:
def get_monthly_data_from_daily(daily_data):
    monthly_data = daily_data[['date_block_num', 'shop_id', 'item_id', 'item_cnt_day']].groupby(
        ['date_block_num', 'shop_id', 'item_id']).sum()
    monthly_data.rename(columns={'item_cnt_day':'item_cnt_month'}, inplace=True)
    monthly_data.reset_index(inplace=True)
    return monthly_data    

In [38]:
monthly_data= get_monthly_data_from_daily(sales_train)
monthly_data['present_in_orig_data'] = 1

In [39]:
master_data_with_test = pd.merge(master_data_with_test, 
                                 monthly_data, 
                                 on=['date_block_num', 'shop_id', 'item_id'],
                                 how='left')

In [40]:
master_data_with_test.fillna(0, inplace=True)
master_data_with_test['item_cnt_month'].clip(0, 20, inplace=True)

In [41]:
master_data_with_test.columns

Index(['date_block_num', 'item_id', 'shop_id', 'item_category_id',
       'item_cnt_month', 'present_in_orig_data'],
      dtype='object')

In [42]:
train_data = master_data_with_test[(master_data_with_test.date_block_num <33)]
validation_data = master_data_with_test[master_data_with_test.date_block_num == 33]
test_data = master_data_with_test[master_data_with_test.date_block_num == 34]

##### Addtitional specific tensorflow functions.

In [43]:
# Train any model, for which a tensorflow estimator API is built. This way, we can test out several models with a one
# line change.
def get_trained_model(model, train_data, outdir=OUTDIR, logging_mode=tf.logging.INFO):
    
    tf.logging.set_verbosity(logging_mode)
    
    # Delete the directory corresponding to metadata of model so as to build it from scratch.
    shutil.rmtree(outdir, ignore_errors=True)

    # Train data for a reasonable number of epochs (100) as default.
    model.train(make_train_input_fn(train_data, num_epochs=100, shuffle=False))
    
    return model

In [44]:
def get_predictions_from_model(model, test_data):
    predictions = model.predict(input_fn=make_prediction_input_fn(test_data)) 
    return [x['predictions'][0] for x in predictions]

##### Train the model and obtain validation score. Also, make score that the predictions are clipped so that they make sense

In [45]:
HIDDEN_UNITS = [20, 20]
#HIDDEN_UNITS = [10, 10, 10]
#HIDDEN_UNITS = [10, 10]
#HIDDEN_UNITS = [5, 5]
#HIDDEN_UNITS = [10]
#HIDDEN_UNITS = [3]
model = get_trained_model(tf.estimator.DNNRegressor(feature_columns=make_feature_cols(),
                                                    hidden_units=HIDDEN_UNITS,
                                                    model_dir=OUTDIR),
                         train_data)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'sample_model_metadata', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x1518f8f28>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
To construct input pipelines, use the `t

INFO:tensorflow:global_step/sec: 143.864
INFO:tensorflow:loss = 142.48198, step = 5501 (0.695 sec)
INFO:tensorflow:global_step/sec: 146.724
INFO:tensorflow:loss = 384.90027, step = 5601 (0.681 sec)
INFO:tensorflow:global_step/sec: 147.396
INFO:tensorflow:loss = 23.728264, step = 5701 (0.678 sec)
INFO:tensorflow:global_step/sec: 147.63
INFO:tensorflow:loss = 92.58806, step = 5801 (0.678 sec)
INFO:tensorflow:global_step/sec: 145.401
INFO:tensorflow:loss = 6.825224, step = 5901 (0.688 sec)
INFO:tensorflow:global_step/sec: 147.005
INFO:tensorflow:loss = 15.963252, step = 6001 (0.681 sec)
INFO:tensorflow:global_step/sec: 140.886
INFO:tensorflow:loss = 27.242786, step = 6101 (0.709 sec)
INFO:tensorflow:global_step/sec: 149.618
INFO:tensorflow:loss = 54.437973, step = 6201 (0.668 sec)
INFO:tensorflow:global_step/sec: 144.948
INFO:tensorflow:loss = 38.52652, step = 6301 (0.690 sec)
INFO:tensorflow:global_step/sec: 142.328
INFO:tensorflow:loss = 113.75784, step = 6401 (0.703 sec)
INFO:tensorflo

INFO:tensorflow:global_step/sec: 150.123
INFO:tensorflow:loss = 50.7584, step = 13801 (0.666 sec)
INFO:tensorflow:global_step/sec: 150.393
INFO:tensorflow:loss = 15.297575, step = 13901 (0.665 sec)
INFO:tensorflow:global_step/sec: 149.223
INFO:tensorflow:loss = 6.9572544, step = 14001 (0.670 sec)
INFO:tensorflow:global_step/sec: 150.255
INFO:tensorflow:loss = 17.184162, step = 14101 (0.665 sec)
INFO:tensorflow:global_step/sec: 149.283
INFO:tensorflow:loss = 25.62546, step = 14201 (0.670 sec)
INFO:tensorflow:global_step/sec: 147.369
INFO:tensorflow:loss = 11.048133, step = 14301 (0.679 sec)
INFO:tensorflow:global_step/sec: 149.096
INFO:tensorflow:loss = 27.265148, step = 14401 (0.671 sec)
INFO:tensorflow:global_step/sec: 146.192
INFO:tensorflow:loss = 71.59902, step = 14501 (0.684 sec)
INFO:tensorflow:global_step/sec: 144.45
INFO:tensorflow:loss = 26.749367, step = 14601 (0.692 sec)
INFO:tensorflow:global_step/sec: 148.038
INFO:tensorflow:loss = 763.9694, step = 14701 (0.675 sec)
INFO:t

INFO:tensorflow:global_step/sec: 145.802
INFO:tensorflow:loss = 69.026665, step = 22001 (0.690 sec)
INFO:tensorflow:global_step/sec: 146.073
INFO:tensorflow:loss = 70.078186, step = 22101 (0.680 sec)
INFO:tensorflow:global_step/sec: 142.824
INFO:tensorflow:loss = 17.827885, step = 22201 (0.700 sec)
INFO:tensorflow:global_step/sec: 147.028
INFO:tensorflow:loss = 72.3453, step = 22301 (0.680 sec)
INFO:tensorflow:global_step/sec: 120.307
INFO:tensorflow:loss = 39.012886, step = 22401 (0.831 sec)
INFO:tensorflow:global_step/sec: 152.429
INFO:tensorflow:loss = 10.303302, step = 22501 (0.656 sec)
INFO:tensorflow:global_step/sec: 146.525
INFO:tensorflow:loss = 221.8522, step = 22601 (0.683 sec)
INFO:tensorflow:global_step/sec: 154.301
INFO:tensorflow:loss = 27.462576, step = 22701 (0.648 sec)
INFO:tensorflow:global_step/sec: 154.041
INFO:tensorflow:loss = 23.907333, step = 22801 (0.649 sec)
INFO:tensorflow:global_step/sec: 155.399
INFO:tensorflow:loss = 326.61035, step = 22901 (0.645 sec)
INF

INFO:tensorflow:loss = 1231.8463, step = 30201 (0.746 sec)
INFO:tensorflow:global_step/sec: 148.262
INFO:tensorflow:loss = 15.5978, step = 30301 (0.674 sec)
INFO:tensorflow:global_step/sec: 135.12
INFO:tensorflow:loss = 863.60077, step = 30401 (0.740 sec)
INFO:tensorflow:global_step/sec: 148.741
INFO:tensorflow:loss = 43.729473, step = 30501 (0.672 sec)
INFO:tensorflow:global_step/sec: 148.106
INFO:tensorflow:loss = 16.616467, step = 30601 (0.675 sec)
INFO:tensorflow:global_step/sec: 143.76
INFO:tensorflow:loss = 400.38928, step = 30701 (0.696 sec)
INFO:tensorflow:global_step/sec: 121.538
INFO:tensorflow:loss = 3.5136673, step = 30801 (0.822 sec)
INFO:tensorflow:global_step/sec: 129.741
INFO:tensorflow:loss = 221.86794, step = 30901 (0.771 sec)
INFO:tensorflow:global_step/sec: 140.354
INFO:tensorflow:loss = 9.712946, step = 31001 (0.712 sec)
INFO:tensorflow:global_step/sec: 140.701
INFO:tensorflow:loss = 9.804976, step = 31101 (0.714 sec)
INFO:tensorflow:global_step/sec: 144.743
INFO:t

...

INFO:tensorflow:global_step/sec: 146.666
INFO:tensorflow:loss = 26.455708, step = 1667101 (0.682 sec)
INFO:tensorflow:global_step/sec: 147.899
INFO:tensorflow:loss = 67.5424, step = 1667201 (0.676 sec)
INFO:tensorflow:global_step/sec: 142.438
INFO:tensorflow:loss = 9.798826, step = 1667301 (0.702 sec)
INFO:tensorflow:global_step/sec: 138.959
INFO:tensorflow:loss = 351.26666, step = 1667401 (0.720 sec)
INFO:tensorflow:global_step/sec: 149.406
INFO:tensorflow:loss = 3.546423, step = 1667501 (0.670 sec)
INFO:tensorflow:global_step/sec: 149.501
INFO:tensorflow:loss = 756.45374, step = 1667601 (0.669 sec)
INFO:tensorflow:global_step/sec: 148.294
INFO:tensorflow:loss = 2.5256748, step = 1667701 (0.674 sec)
INFO:tensorflow:global_step/sec: 144.547
INFO:tensorflow:loss = 288.03903, step = 1667801 (0.692 sec)
INFO:tensorflow:global_step/sec: 148.375
INFO:tensorflow:loss = 24.313694, step = 1667901 (0.674 sec)
INFO:tensorflow:global_step/sec: 152.719
INFO:tensorflow:loss = 404.19263, step = 1668

INFO:tensorflow:global_step/sec: 137.884
INFO:tensorflow:loss = 12.143994, step = 1675201 (0.725 sec)
INFO:tensorflow:global_step/sec: 138.369
INFO:tensorflow:loss = 82.85617, step = 1675301 (0.723 sec)
INFO:tensorflow:global_step/sec: 140.815
INFO:tensorflow:loss = 19.835625, step = 1675401 (0.710 sec)
INFO:tensorflow:global_step/sec: 135.291
INFO:tensorflow:loss = 50.298695, step = 1675501 (0.739 sec)
INFO:tensorflow:global_step/sec: 130.591
INFO:tensorflow:loss = 16.554138, step = 1675601 (0.766 sec)
INFO:tensorflow:global_step/sec: 131.954
INFO:tensorflow:loss = 19.21362, step = 1675701 (0.758 sec)
INFO:tensorflow:global_step/sec: 137.714
INFO:tensorflow:loss = 4.6398287, step = 1675801 (0.726 sec)
INFO:tensorflow:global_step/sec: 139.52
INFO:tensorflow:loss = 15.258291, step = 1675901 (0.717 sec)
INFO:tensorflow:global_step/sec: 135.367
INFO:tensorflow:loss = 2.2437754, step = 1676001 (0.739 sec)
INFO:tensorflow:global_step/sec: 139.291
INFO:tensorflow:loss = 3.594035, step = 1676

INFO:tensorflow:global_step/sec: 149.308
INFO:tensorflow:loss = 4.5294333, step = 1683301 (0.670 sec)
INFO:tensorflow:global_step/sec: 151.249
INFO:tensorflow:loss = 13.782269, step = 1683401 (0.661 sec)
INFO:tensorflow:global_step/sec: 143.312
INFO:tensorflow:loss = 322.0289, step = 1683501 (0.698 sec)
INFO:tensorflow:global_step/sec: 153.439
INFO:tensorflow:loss = 16.848015, step = 1683601 (0.652 sec)
INFO:tensorflow:global_step/sec: 149.939
INFO:tensorflow:loss = 364.30157, step = 1683701 (0.667 sec)
INFO:tensorflow:global_step/sec: 149.765
INFO:tensorflow:loss = 14.2523575, step = 1683801 (0.668 sec)
INFO:tensorflow:global_step/sec: 150.385
INFO:tensorflow:loss = 92.462746, step = 1683901 (0.665 sec)
INFO:tensorflow:global_step/sec: 151.989
INFO:tensorflow:loss = 3.1619332, step = 1684001 (0.658 sec)
INFO:tensorflow:global_step/sec: 138.24
INFO:tensorflow:loss = 521.6552, step = 1684101 (0.723 sec)
INFO:tensorflow:global_step/sec: 149.78
INFO:tensorflow:loss = 6.6140785, step = 168

INFO:tensorflow:global_step/sec: 143.946
INFO:tensorflow:loss = 4.7155056, step = 1691401 (0.695 sec)
INFO:tensorflow:global_step/sec: 145.732
INFO:tensorflow:loss = 9.556384, step = 1691501 (0.686 sec)
INFO:tensorflow:global_step/sec: 147.954
INFO:tensorflow:loss = 141.28157, step = 1691601 (0.676 sec)
INFO:tensorflow:global_step/sec: 148.419
INFO:tensorflow:loss = 5.4789248, step = 1691701 (0.674 sec)
INFO:tensorflow:global_step/sec: 149.673
INFO:tensorflow:loss = 158.50563, step = 1691801 (0.669 sec)
INFO:tensorflow:global_step/sec: 150.902
INFO:tensorflow:loss = 5.369833, step = 1691901 (0.662 sec)
INFO:tensorflow:global_step/sec: 152.046
INFO:tensorflow:loss = 4.02516, step = 1692001 (0.658 sec)
INFO:tensorflow:global_step/sec: 151.86
INFO:tensorflow:loss = 56.949066, step = 1692101 (0.659 sec)
INFO:tensorflow:global_step/sec: 151.624
INFO:tensorflow:loss = 15.48286, step = 1692201 (0.659 sec)
INFO:tensorflow:global_step/sec: 152.675
INFO:tensorflow:loss = 12.468116, step = 169230

KeyboardInterrupt: 

In [191]:
model_predictions = get_predictions_from_model(model, validation_data)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from sample_model_metadata/model.ckpt-1200
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


In [192]:
def get_rmse(model_predictions, validation_data):
    actual_vals = np.clip(validation_data.item_cnt_month.values, 0, 20)
    return np.sqrt(mean_squared_error(actual_vals, np.clip(model_predictions, 0, 20)))

In [193]:
get_rmse(model_predictions, validation_data)

2.6254074716990776